<a href="https://colab.research.google.com/github/JamesPeralta/Machine-Learning-Algorithms/blob/master/Miscellaneous/Keras%20Callbacks/Keras_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Callbacks
### A callback is an object (a class instance implementing specific methods) that is passed to the model in the call to fit and that is called by the model at various points during training.
### It has access to all the available data about the state of the model and its performance, and it can take action: interrupt training, save a model, load a different weight set, or otherwise alter the state of the model.

In [1]:
import keras

Using TensorFlow backend.


## Model checkpointing — Saving the current weights of the model at different points during training.
## Early stopping — Interrupting training when the validation loss is no longer improving (and of course, saving the best model obtained during training).
### Use the EarlyStopping callback to interrupt training once a target metric being monitored has stopped improving for a fixed number of epochs. For instance, this callback allows you to interrupt training as soon as you start overfitting, thus avoiding having to retrain your model for a smaller number of epochs

In [0]:
# Callbacks are passed to the model as a list in the fit method
callbacks_list = [
        keras.callbacks.EarlyStopping( # Interrupts training when improvment stops
            monitor='acc', # Monitors the model's validation accuracy
            patience=1, # Number of epochs to wait before stopping
        ), 
        keras.callbacks.ModelCheckpoint( # Saves the current weights after every epoch 
            filepath='my_model.h5', # Path to the destination model file
            monitor='val_loss',
            save_best_only=True,) # won't overwrite the model file unless val_loss has improved
]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc']) # Since we are monitoring accuracy it should be part of the models metrics

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list, # Pass in the Callback list
          validation_data=(x_val, y_val)) # since we are will also monitor validation_data

## Dynamically adjusting the value of certain parameters during training — Such as the learning rate of the optimizer.
### Use this callback to reduce the learning rate when the validation loss has stopped improving. Reducing or increasing the learning rate in case of a loss plateau is is an effective strategy to get out of local minima during training.

In [0]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss',
      factor=0.1,
      patience=10,
    )
]

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

## Writing Custom Callbacks
### If you need to take a specific action during training that isn’t covered by one of the built-in callbacks, you can write your own callback. Callbacks are implemented by subclassing the class keras.callbacks.Callback.
### You can implement any one of these methods:
```
on_epoch_begin <-- Called at the start of every epoch 
on_epoch_end   <-- Called at the end of every epoch

on_batch_begin <-- Called right before processing each batch
on_batch_end   <-- Called right after processing each batch

on_train_begin <-- Called at the start of training
on_train_end   <-- Called at the end of training
```
### These methods all are called with a logs argument, which is a dictionary containing information about the previous batch, epoch, or training run: training and validation metrics, etc. The call back also has access to the following attributes
*   self.model—The model instance from which the callback is being called
*   self.validation_data—The value of what was passed to fit as validation data



### Ex) Write a custom callback that saves to disk (as Numpy arrays) the activations of every layer of the model at the end of every epoch, computed on the first sample of the validation set

In [0]:
class ActivationLogger(keras.callbacks.Callback):
  
  def set_model(self, model):
    self.model = model # Called by the parent model before training, to inform the callback of what model will be calling it
    layer_outputs = [layer.output for layer in model.layers]
    self.activations_model = keras.models.Model(model.input,
                                                layer_outputs)
    
  def on_epoch_end(self, epoch, logs=None):
    if self.validation_data is None:
        raise RuntimeError('Requires validation_data.')
    validation_sample = self.validation_data[0][0:1]
    activations = self.activations_model.predict(validation_sample)
    f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
    np.savez(f, activations)
    f.close()
    